In [1]:
%pip install newsapi-python
%pip install requests
%pip install newsapi-python
%pip install matplotlib
%pip install pandas
%pip install bs4 
%pip install numpy

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring instal

In [8]:
import pprint
import requests
from newsapi import NewsApiClient
import datetime
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
from datetime import datetime as dt
from datetime import timedelta

secret = '629525a47ff6458789d19a3c489419fa'
  
# Define the endpoint
url = 'https://newsapi.org/v2/everything'



# Get Articles for one player
First Function get_articlesplayer takes a player, language and a date until which the article data should be created as an input. 
The Function then calls the api and collects all articles from the current date until the input date. The articles are then returned as a Dataframe.

In [3]:
def get_articlesplayer(player, language, date_to):

    # Set the url from newsapi
    url = 'https://newsapi.org/v2/everything'

    # Create date_form which is the current date
    today = datetime.date.today()
    date_from = today.strftime("%Y-%m-%d")


    # Specify the query and number of returns
    parameters = {
        'q': player, # query phrase
        'pageSize': 100,  # maximum is 100
        'apiKey': secret, # your own API key
        'sortBy':'publishedAt',
        'from': date_from,
        'to': date_to, 
        'language': language
    }

    # get the response json
    response = requests.get(url, params = parameters)

    # Parse the JSON response and extract article URLs
    if response.ok:
        data = response.json()
        urls = [article['url'] for article in data['articles']]
    else:
        print('Error: Request failed with status', response.status_code)

    # Scrape the full article content for each URL and store in a dataframe
    articles = []
    for url in urls:
        # Make a request to the article URL
        response = requests.get(url)

        # Parse the HTML response and extract the article content
        if response.ok:
            soup = BeautifulSoup(response.content, 'html.parser')
        
            content = '\n'.join([p.text.strip() for p in soup.find_all('p')])
            articles.append({'content': content})
        else:
            print('Error: Request failed with status', response.status_code)


    # Convert the list of articles to a Pandas dataframe
    df = pd.DataFrame(articles)

    return(articles)




# Get Articles for multiple Player and multiple Languages
The get_players_df takes a list of Players, a list of languages and a date until the articles should be collected as an input. 
The function then loops over the player and langauge list and calls the get_articlesplayer on all of them. After that a Datframe will be created consisting of the articles, playernames and the given language. 

In [4]:
def get_players_df(playerlist, languagelist, date_to):

    # Create an empty Datframe where all player data is stored
    df_all_players = pd.DataFrame(columns=['data', 'player', 'language'])

    # Loop over all players in the player list 
    for player in playerlist:

    # Loop over all languages in the languages list 
        for language in languagelist:

            # Call the get_articlesplayer to get the articles for a given player and language
            articles_arr = get_articlesplayer(player, language, date_to)

            # Create an array with the current language of the same size as the articles data
            language_arr = np.full((len(articles_arr)), language)

            # Create an array with the current player name of the same size as the articles data
            player_arr = np.full((len(articles_arr)), player)

            # Create a datframe with the articles, playername and language
            df_one_lang = pd.DataFrame({'data': articles_arr, 'player': player_arr,'language': language_arr})

            # Add the data of the iteration to the prior data 
            df_all_players = pd.concat([df_all_players, df_one_lang])

    return df_all_players



# Function to get the Date where a file was last updated
The Function takes a file as an input and deliveres the last updated time of the file as a String

In [9]:
def get_date_of_file(file):
    # get the last time the file was updated
    creation_time = os.stat(file)
    last_modified_time = creation_time.st_mtime

    # convert the float time into readable format
    seconds = int(last_modified_time)
    microseconds = int((last_modified_time - seconds) * 1000000)
    dta = dt.fromtimestamp(seconds).replace(microsecond=microseconds)
    date_string = dta.strftime('%m-%d-%Y')
    
    #date_obj = dta.strptime(date_string, '%m-%d-%Y')
    # Add one day to the datetime object
    #new_date_obj = date_obj + timedelta(days=1)

    # Convert the new datetime object to a string in the desired format
    #date_to = new_date_obj.strftime('%m-%d-%Y')

    return date_string


# Function to update a given file 
Within the Function a file which needs to be updated is taken as an input. The Function collects all player data from the last day of updating and then inserts the data 

In [6]:
def updateplayerdata(file):
    # create the list of players which should be selected
    playerlist = ['Mitchel Bakker', 'Jeremie Frimpong', 'Moussa Diaby', 'Jonathan Tah', 'Piero Hincapie', 'Exequiel Palacios', 'Mykhaylo Mudryk'] 
    # create a list of languages 
    languagelist = ['en', 'de', 'es']

    # call the get_players_df function on the players, languages and the last time the file was updated 
    df_new = get_players_df(playerlist, languagelist, get_date_of_file(file))

    # Open the CSV file in append mode and write the DataFrame
    with open(file, 'a', newline='') as f:
        df_new.to_csv(f, header=f.tell()==0, index=False)

    return df_new

In [7]:
file = '/Users/kevingiesen/Library/Mobile Documents/com~apple~CloudDocs/BIPM Master/Semester 2/TWSM/TWSM Project/SS23-BIPM-Analytics-Lab---Group-4-repository/data_files/all_data_v2.csv'

updateplayerdata(file)

,data,player,language
